In [58]:
import os
import csv
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from numpy import exp, loadtxt, pi, sqrt
import glob
import json
from lmfit import Model, Parameters, fit_report, minimize, Minimizer
from lmfit.models import GaussianModel, LinearModel, ConstantModel
os.chdir(r'C:\Users\kiat y_c\Desktop\Warwick Main\Year 3\Synchrotron & Neutron Experiments\SXRD Experiments\DIAMOND Round 1\MG27288\103423-pilatus2M-files\103423_ascii') #working directory
filenames = glob.glob( '*dat' ) #use the .dat file that I have sent you

#function
for filename in filenames:
    data = loadtxt(filename)
    x = data[:, 0]
    y = data[:, 1]
    
    define_mod = GaussianModel (prefix='g1_')
    line_mod = ConstantModel (prefix='line_')
       
    pars = define_mod.guess(y, x=x)
    pars += line_mod.guess (y, x=x)
    
    mod = define_mod + line_mod
    
    out = mod.fit(y, pars, x=x)    
 
    #writing the output to a csv file     
    #with open('report_'+filename+'.txt','w') as f:
        #f.write(out.fit_report(min_correl=0.25))
        
    #plotting the graph
    plt.plot(x, y, '.')
    plt.plot(x, out.best_fit, 'r', label='Best fit')
    
    comps = out.eval_components()
    plt.plot(x, comps['g1_'], 'b--', label='Gaussian component')
    plt.axhline(comps ['line_'], label='Line component')
    plt.legend(loc='best')
    plt.savefig('figure_'+filename+'.jpg')
    plt.clf()
    
    y = json.loads(out.params.dumps())
    
    x = pd.DataFrame(y['params'])
    
    x.columns = ['Attribute','Value','C','D','E','F','G','Error','I','J','K']
    
    x = x.drop(columns = ['C','D','E','F','G','I','J','K'])
    
    x_transposed = x.T
    
    x_transposed = x_transposed.drop(x_transposed.index[0])
    
    df = x_transposed
    df2 = df.join(df.shift(-1).add_prefix('Error_'))
    df2 = df2.drop(['Error'])

    df2.to_csv(filename+'.csv')

<Figure size 432x288 with 0 Axes>

In [56]:
x

,Attribute,Value,Error
0,g1_amplitude,0.000468,0.000041
1,g1_center,5.719329,0.000364
2,g1_sigma,0.003787,0.000370
3,g1_fwhm,0.008918,0.000872
4,g1_height,0.049261,0.004120
5,line_c,0.070688,0.000646
